In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.tracers.stdout import ConsoleCallbackHandler

In [ ]:
os.environ['OPENAI_API_KEY'] = "yourServiceKey"

llm = ChatOpenAI(
  temperature=0.1, # 창의성 (0.0 ~ 2.0)
  max_tokens=2048,
  model_name="gpt-3.5-turbo"
)

question = "세종대왕이 누구인지 설명해주세요"

result = llm.invoke(question)
print(result.content)

세종대왕은 조선시대 4대 왕 중 하나로, 조선시대의 제4대 군주이자 문신이다. 세종대왕은 1418년에 태어나 1450년에 왕위에 오르고, 1418년부터 1450년까지 32년 동안 통치했다. 그는 조선시대의 가장 위대한 왕 중 하나로 꼽히며, 한글을 창제하고 과학기술, 문화, 예술 등을 발전시키는 데 큰 기여를 했다. 세종대왕은 또한 국방력을 강화하고 행정체제를 현대화하는 등 다양한 분야에서 혁신을 이끌었다. 그의 통치는 조선시대의 전성기를 이루는 데 큰 역할을 했다.


In [3]:
template = "{who}가 누구인지 설명해주세요"

prompt = PromptTemplate(
  template=template, input_variables=['who']
)

print(prompt)

input_variables=['who'] input_types={} partial_variables={} template='{who}가 누구인지 설명해주세요'


In [5]:
llm_chain = prompt | llm

result = llm_chain.invoke({"who":"이순신 장군"}, config={"callbacks": [ConsoleCallbackHandler()]})
print(result.content)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "who": "이순신 장군"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "who": "이순신 장군"
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: 이순신 장군가 누구인지 설명해주세요"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] [5.17s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "이순신 장군은 조선시대의 무신이자 미신으로 유명한 역사적 인물입니다. 그는 조선시대 중기에 살았으며, 조선의 해상군을 이끌었던 뛰어난 장군으로서 유명합니다. 이순신은 일본의 침략을 막기 위해 수많은 전투를 치뤘고, 특히 명량 해전에서 일본의 강력한 함대를 물리쳐 조선의 승리를 이끌었습니다. 이순신은 그의 뛰어난 전략과 냉정한 판단력으로 많은 사람들에게 존경받고 있으며, 한국 역사상 가장 위대한 장군 중 한 명으로 평가받고 있습니다.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
  

In [6]:
template = """
아래는 사람과 AI의 친근한 대화입니다. AI의 이름은 위키북스봇입니다. 대화 문맥을 바탕으로 친절한 답변을 진행하세요.

Current Conversation:
{history}

Human: {input}
AI:
"""

prompt = PromptTemplate(
  template=template, input_variables=['history', 'input']
)

llm = ChatOpenAI(model_name="gpt-3.5-turbo")
chain = prompt | llm

In [7]:
store = {}
session_id = "test"

if session_id not in store:
  store[session_id] = ChatMessageHistory()

session_history = store[session_id]

with_message_history = RunnableWithMessageHistory(
  chain,
  lambda session_id: session_history,
  input_messages_key="input",
  history_messages_key="history"
)

result = with_message_history.invoke(
  {"input": "당신은 어디에서 만들었습니까?"},
  config={"configurable":{"session_id":"test"}},
)

print(result.content)

저는 위키북스에서 만들어진 AI입니다. 위키북스는 자동으로 책을 요약하는 인공지능을 개발하는 기업이에요.


In [8]:
result = with_message_history.invoke(
  {"input":"푸른 바다를 주제로 감성적이고 짧은 시를 하나 지어주세요"},
  config={"configurable": {"session_id":"test"}}
)

print(result.content)

파도 타고 춤추는 바다
햇살 따라 춤 추는 파도
물결 위 춤추는 노래
바다의 푸른 노래


In [9]:
reuslt = with_message_history.invoke(
  {"input":"석양을 주제로도 해줘"},
  config={"configurable":{"session_id":"test"}}
)

In [10]:
print(reuslt.content)

태양이 지는 저녁\n하늘을 물들이는 석양\n따뜻한 빛에 안기는 순간\n마음을 감싸는 평화와 아름다움


In [11]:
print(store)

{'test': InMemoryChatMessageHistory(messages=[HumanMessage(content='당신은 어디에서 만들었습니까?', additional_kwargs={}, response_metadata={}), AIMessage(content='저는 위키북스에서 만들어진 AI입니다. 위키북스는 자동으로 책을 요약하는 인공지능을 개발하는 기업이에요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 93, 'total_tokens': 144, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BpyeSvFtVbSm125hDNB8vjRCyCdpo', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--36bf20e4-42e6-43e4-be06-4f729dde3340-0', usage_metadata={'input_tokens': 93, 'output_tokens': 51, 'total_tokens': 144, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessage(content='